# Quantile mapping of TCLV intensity

This notebook is used to map the intensity of TCLVs to the observed historical distribution of intensity. 

We work with the central pressure deficit $\Delta p_c = p_{oci} - p_{c}$, where $p_{oci}$ is the pressure of the outermost closed isobar (sometimes also referred to as environmental pressure $p_{env}$) and $p_c$ is the minimum central pressure. We use $\Delta p_c$, as this is the variable used in TCRM to model intensity. $p_c$ is also more widely recorded in the Australian region, for reasons that I'll not go into here. 

Irrespective of the variable chosen, regionally downscaled climate models, even at 10 km grid spacing, do not sufficiently resolve the dynamics that drive TC intensity. To accurately simulate maximum intensity (represented by surface wind speed) arguably requires a convection-allowing model, which usually entails grid spacings less than 2 km (e.g. Gentry and Lackmann, 2010; Walsh _et al_., 2013). Such modelling efforts are beyond the scope of this project, so we rely on scaling the intensity to address the issue.

We use a quantile mapping approach to adjust the $\Delta p_c$ of the TCLVs extracted for future climate projections which does not _a priori_ assume a stationary relationship in the scaling function (as we have previously done, for example, in Arthur and Woolf, 2015). Cannon _et al._ (2015) describe a method for bias correction of climate variables that conserves relative changes in quantiles between current and future climate regimes. Called Quantile Delta Mapping (QDM), the method ensures that climate sensitivity of the underlying climate model remains unaffected by the correction process. The concepts described in Cannon _et al._ have been applied to a range of other climate variables - e.g. Bhatia _et al._ (2019) have applied QDM to intensity changes of simulated TCs in high-resolution simulations.

Changes in frequency and track behavoiur - both key factors in understanding likelihood of extreme winds - are drawn directly from the TCLV data and are examined in a separate notebook.

In [ ]:
%matplotlib inline

import os
from os.path import join as pjoin
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

import re
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import cartopy.crs as ccrs
import cartopy.feature as feature
import statsmodels.api as sm
from datetime import timedelta, datetime

import shapely.geometry as sg
from shapely.geometry import box as sbox
from shapely.geometry import LineString

from scipy.optimize import curve_fit
import scipy.stats as stats

# From TCRM codebase
from Utilities.loadData import maxWindSpeed

from builtins import str
sns.set_style('whitegrid')
sns.set_context("poster")

This section defines a number of functions that are used to load and filter the track data generated by the regional downscaling simulations. 

Filters are applied to eliminate spurious tracks that have a short lifetime. These spurious tracks are picked up by the detection and tracking method, but are not persisted for a sufficient time period (36 hours), which leads to an anomolous occurence of TCLVs around the Solomon Islands in some of the model output. 

A spatial filter is used to select only those tracks that pass through the study region - in this case Queensland and the Coral Sea. The regional downscaling was focused over the Queensland region (using a cubic-conformal grid configuration), so the marginal regions are unlikely to resolve the dynamics of the inner core of TCLVs. As the grid spacing of the computational grid increases, the fine scale features that control intensity of TCLVs will not be as well resolved, which will affect the distribution of intensity in simulated events.

In [ ]:
def load_track_file(filename):
    """
    Load a TCLV file into a :class:`pandas.DataFrame`, and add a field 
    representing the age of each TCLV in hours, and the pressure difference.
    
    :param str filename: Path to a TCLV data file
    
    :returns: :class:`pandas.DataFrame`
    """
    # This assumes the format of the TCLV files is identical
    columns = ['num', 'year', 'month', 'day', 'hour', 'lon', 'lat',
               'pmin', 'vorticity', 'vmax', 'tanomsum','tanomdiff',
               'pmslanom', 'poci', 'reff','ravg','asym']
    df = pd.read_csv(filename, delimiter=' ', skipinitialspace=True,
                     names=columns, parse_dates={'datetime':[1,2,3,4]},
                     keep_date_col=True, 
                     dtype={'year':int, 'month':int, 'day':int})
    df['dt'] = df.groupby('num')['datetime'].apply(lambda x: x.diff())
    df['dt'] = df['dt'].transform(lambda x: x.total_seconds())

    df['age'] = df.groupby('num')['dt'].apply(np.cumsum).fillna(0)/3600.
    # Throw in the pressure deficit for good measure:
    df['pdiff'] = df['poci'] - df['pmin']
    # And normalised intensity. This is the intensity at any given time,
    # dividied by the lifetime maximum intensity for each unique event
    df['ni'] = df.pdiff / df.groupby('num').pdiff.transform(np.max)

    return df

def filter_tracks(df, start_year=1980, end_year=2010, zeta=0, age=36):
    """
    Takes a `DataFrame` and filters on the basis of a prescribed vorticity 
    threshold, lifetime and a given time period.
    
    :param df: :class:`pandas.DataFrame` that holds the TCLV data
    :param int start_year: Starting year of the time period to filter
    :param int end_year: End year of the period to filter
    :param float zeta: Vorticity threshold to filter the TCLV data. 
                       This can be a positive value, as we filter on the
                       absolute value of the field.
    :param int age: Minimum age of the TCLVs in hours
    
    """
    tracks = df.groupby('num')
    filterdf = tracks.filter(lambda x: (x['datetime'].dt.year.min() >= start_year) &\
                                       (x['datetime'].dt.year.max() <= end_year) &\
                                       (x['age'].max() >= age) &\
                                       (np.abs(x['vorticity'].min()) > zeta))
    return filterdf

def filter_tracks_domain(df, minlon=90, maxlon=180, minlat=-40, maxlat=0):
    """
    Takes a `DataFrame` and filters on the basis of whether the track interscts
    the given domain, which is specified by the minimum and maximum longitude and 
    latitude.
    
    NOTE: This assumes the tracks and bounding box are in the same geographic 
    coordinate system (i.e. generally a latitude-longitude coordinate system). 
    It will NOT support different projections (e.g. UTM data for the bounds and
    geographic for the tracks).
    
    NOTE: This doesn't work if there is only one point for the track. 
    
    :param df: :class:`pandas.DataFrame` that holds the TCLV data
    :param float minlon: minimum longitude of the bounding box
    :param float minlat: minimum latitude of the bounding box
    :param float maxlon: maximum longitude of the bounding box
    :param float maxlat: maximum latitude of the bounding box
    """
    domain = sbox(minlon, minlat, maxlon, maxlat, ccw=False)
    tracks = df.groupby('num')
    tempfilter = tracks.filter(lambda x: len(x) > 1)
    tempfilter.head()
    filterdf = tempfilter.groupby('num').filter(lambda x: LineString(zip(x['lon'], x['lat'])).intersects(domain))
    return filterdf


def calculateMaxWind(df, dtname='ISO_TIME'):
    """
    Calculate a maximum gust wind speed based on the central pressure deficit and the 
    wind-pressure relation defined in Holland (2008). This uses the function defined in 
    the TCRM code base, and simply passes the correct variables from the data frame
    to the function
    
    This returns a `DataFrame` with an additional column (`vmax`), which represents an estimated
    0.2 second maximum gust wind speed.
    """
    idx = df.num.values
    varidx = np.ones(len(idx))
    varidx[1:][idx[1:]==idx[:-1]] = 0
    
    dt = (df[dtname] - df[dtname].shift()).fillna(pd.Timedelta(seconds=0)).apply(lambda x: x / np.timedelta64(1,'h')).astype('int64') % (24*60)
    df['vmax'] = maxWindSpeed(varidx, dt.values, df.lon.values, df.lat.values,
                              df.pmin.values, df.poci.values, gustfactor=1.223)
    return df

In [ ]:
path = "../data/tclv/"
regex = r'all_tracks_(.+)_(rcp\d+)\.dat'
labels = ['TD', 'TC1', 'TC2', 'TC3', 'TC4', 'TC5']
catpal = sns.blend_palette([(0.000, 0.627, 0.235), (0.412, 0.627, 0.235), 
                            (0.663, 0.780, 0.282), (0.957, 0.812, 0.000), 
                            (0.925, 0.643, 0.016), (0.835, 0.314, 0.118),
                            (0.780, 0.086, 0.118)], 6)
dist = stats.lognorm

The variables are referenced in the following way::

* `s` = simulated
* `ref` = reference time period (1981-2010)
* `fut` = future time period (2021-2040, 2041-2060, 2061-2080 or 2081-2100)
* `obstc` = observed TC events (using IBTrACS, 1981-2010)
* `b` = bias-corrected

Firstly load the IBTrACS data and extract the relevant tracks for the region of interest. We retain only those tracks that have a vaild $\Delta p$ value and enter the simulation domain.

In [ ]:
best = pd.read_csv('../data/ibtracs.since1980.list.v04r00.csv', 
                       skiprows=[1],
                       usecols=[0,6,8,9,11,113],
                       na_values=[' '],
                       parse_dates=[1])
best.rename(columns={'SID':'num', 'LAT': 'lat', 'LON':'lon', 'WMO_PRES':'pmin', 'BOM_POCI':'poci'}, inplace=True)
best = best[best.poci.notnull() & best.pmin.notnull()]
best['pdiff'] = best.poci - best.pmin
best = best[best.pdiff > 0]
obstc = filter_tracks_domain(best, 135, 160, -25, -10)

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines(resolution='10m', color='black', linewidth=1)
ax.add_feature(feature.BORDERS)
gl = ax.gridlines(linestyle=":", draw_labels=True)
ax.add_feature(feature.LAND, zorder=0)
for k, v in obstc.groupby('num'):
    ax.plot(v['lon'], v['lat'])

ax.add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                fill=False, edgecolor='k', linewidth=3,
                                transform=ccrs.PlateCarree())
             )
None
plt.savefig(pjoin(path, "observed_tracks.png"), bbox_inches='tight')

The figure above plots all tracks that enter the selected domain (135-170$^\circ$E, 5-35$^\circ$S), and have a $p_{oci}$ defined in the historical record. This variable was not widely reported, and then only those events that are in the Bureau of Meteorology's area of responsibility have this field (this leads to the artificial boundary at 160$^\circ$E). 

### Defining the quantile delta mapping function

Here is where we develop the QDM algorithm. The algorithm is based on the QDM described by Cannon _et al._ (2015), where the relative change in quantiles between a reference period and the future period are preserved. 

$\delta_{fut} = \dfrac{Q_{fut}}{F_{ref}^{-1}\left[ F_{fut} ( Q_{fut} ) \right]} $ 

$Q_{futb} = F_{obs}^{-1} \left[ F_{fut} (Q_{fut}) \right] \times \delta_{fut} $

$\delta_{fut}$ is the relative change in the quantiles between the simulated reference data ('sreftclv') and the simulated future data ('sfuttclv'). $Q_{fut}$ is the quantile of the simulated future data. $F_{fut}$ and $F_{ref}^{-1}$ are a CDF of the simulated future data and an inverse CDF of the simulated reference data respectively. Finally, $F_{obs}^{-1}$ is the inverse CDF of the observed data, and $Q_{futb}$ are the corrected quantiles of the future data.

In this framework, the algorithm is independent of the selection of distribution, which would be data-dependent, and more generally specific to the variable that is being corrected. We begin by fitting a lognormal distribution to the $\Delta p_c$ values in each of the observed, reference and future collections. 

For our analysis, the reference period is 1981-2010. Future periods will be based on 20-year time slices centred on 2030, 2050, 2070 and 2090. This will give four different corrections for each model and RCP.  

The resulting bias-corrected $\Delta p_c$ values will be applied to the TCLV data, which will in turn be used as input to hazard simulations for the reference and future climate periods. As a final check, we will calculate a maximum wind speed based on Holland's (2008) wind pressure relation, and explore changes in the occurrence of severe TCs (i.e. category 3-5 TCs based on the Bureau of Meteorology's TC intensity scale). 

In [ ]:
def qdm(vobs, vref, vfut, dist=stats.lognorm):
    """
    Calculate the quantile delta mapping for a collection of simulated data. 
    
    This function is based on the formulation described in Cannon _et al._ (2015) and Heo _et al._ (2019).
    
    :param vobs: numpy.array of observed values
    :param vref: numpy.array of reference period values (simulated)
    :param vfut: numpy.array of future period values (simulated)
    :param func dist: Distribution function to use. This must have `fit`, `pdf`, `cdf` and `ppf` methods defined, 
                      as used in `scipy.stats.rv_continuous` distributions. Default is `scipy.stats.lognorm`.
                      
    :returns: `vfutb` a numpy.array of bias corrected future values. 
    """
    
    pobs = dist.fit(vobs, loc=0, scale=1)
    pref = dist.fit(vref, loc=0, scale=1)
    pfut = dist.fit(vfut, loc=0, scale=1)
    
    # CDF of future, at the value of the future data points
    Fsf = dist.cdf(vfut, *pfut)
    # Inverse cdf of reference period distribution, evaluated at future CDF
    # values
    invFsr = dist.ppf(Fsf, *pref)
    
    # Relative change in values
    delta = vfut / invFsr
    vfutb = dist.ppf(Fsf, *pobs) * delta
    
    return vfutb

First we explore the distribution of $\Delta p_c$ for the reference period (1981-2010) in each of the models, and the best track archive. The figure below shows the histogram of $\Delta p_c$, with a fitted lognormal distribution to each. 

In [ ]:
refdata = {}
regex = r'all_tracks_(.+)_(rcp\d+)\.dat'
ens45 = []
ens85 = []
for i, fname in enumerate(os.listdir(path)):
    if fname=="all_tracks_ERAIntQ_rcp85.dat":
        continue
    m = re.match(regex, fname)
    try:
        model, rcp = m.group(1, 2)
    except AttributeError: 
        # Catch the case where there are other files or folders in the directory
        continue
    
    filename = pjoin(path, fname)
    df = load_track_file(filename)
    df = filter_tracks(df)
    df = filter_tracks_domain(df, 135, 160, -25, -10)
    if rcp.upper() == 'RCP45':
        ens45.append(df)
    else:
        ens85.append(df)
    label = f"{model} {rcp.upper()}"
    refdata[label] = df

In [ ]:
refdata['ENS RCP45'] = pd.concat(ens45, ignore_index=True)
refdata['ENS RCP85'] = pd.concat(ens85, ignore_index=True)

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
bins = np.arange(0, 100, 5)
refparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, df) in enumerate(refdata.items()):
    sns.distplot(df.pdiff, ax=ax[i], kde=False, norm_hist=True)
    model, rcp = m.split(' ')
    popt = stats.lognorm.fit(df.pdiff, loc=0, scale=1)
    refparams = refparams.append({'Model':model, 'RCP':rcp, 
                                  'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                ignore_index=True)
    fitline = stats.lognorm.pdf(np.arange(0, 101), *popt, )
    ax[i].plot(np.arange(0,101), fitline, color='r')
    ax[i].set_title("{0}\n({1:.4f}, {2:.4f}, {3:.4f})".format(m, *popt))
 

# Add the IBTrACS as reference
#sns.distplot(obstc.pdiff, fit=stats.lognorm, kde=False, color='red', ax=ax[-1])
obsparams = stats.lognorm.fit(obstc.pdiff, loc=0, scale=1)
#ax[-1].set_title("IBTrACS\n({0:.4f}, {1:.4f}, {2:.4f})".format(*obsparams))
fig.tight_layout()
plt.savefig(pjoin(path, "reference_distribution.png"), bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,5), sharey=True)
x = np.arange(0, 100)

for index, row in refparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if row.Model == 'ENS':
        ax[0].plot(x, fitline, color='b')
    else:
        ax[0].plot(x, fitline, color='k')
        
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in refparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Reference')
    elif row.Model == 'ENS':
        ax[1].plot(x, fitline, color='b', label='Ensemble')
    else:
        ax[1].plot(x, fitline, color='k')
        
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[0].set_ylabel("Probability")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None
plt.savefig(pjoin(path, 'uncorrected_distribution_by_RCP_reference.png'), bbox_inches='tight')

Many of the models produce $\Delta p_c$ distributions that follow a lognormal distribution, but all are markedly different from the IBTrACS distribution (which is also a lognormal distribution). All the models have a lower mean compared to the IBTrACS data, indicative that the models are unable to replicate the larger $\Delta p_c$ values in the observed record. There appears to be two families of models, but I'm yet to explore which is which, and further to that, whether these are consistent between the reference period and the future period (but we could do that based on figures below).

However, that the distributions are all nominally lognormal, as is the observed distribution, gives confidence that the QDM approach will be suitable.

In [ ]:
refparams.groupby(['RCP']).agg('mean')

There's little difference between the mean distributions for RCP8.5 and RCP4.5 models, based on the reference period. Again, this is an expected result, as the models should be simulating a common baseline period (the climate forcing in the reference period is common across the CMIP models [ref])

We now load up the data for a future period and look at the distributions of $\Delta p_c$ for the future data. For this demonstration, the future period is 2081-2100.

In [ ]:
start=2081
end=2100
futdata = {}
ens45 = []
ens85 = []
for i, f in enumerate(os.listdir(path)):
    # Exclude the ERA Interim data
    if f=="all_tracks_ERAIntQ_rcp85.dat":
        continue
    m = re.match(regex, f)
    try:
        model, rcp = m.group(1, 2)
    except AttributeError:
        continue
    
    filename = pjoin(path, f)
    df = load_track_file(filename)
    df = filter_tracks(df, start_year=start, end_year=end)
    df = filter_tracks_domain(df, 135, 160, -25, -10)
    if rcp.upper() == 'RCP45':
        ens45.append(df)
    else:
        ens85.append(df)
    label = "{0} {1}".format(model, rcp.upper())
    futdata[label] = df
    
futdata['ENS RCP45'] = pd.concat(ens45, ignore_index=True)
futdata['ENS RCP85'] = pd.concat(ens85, ignore_index=True)

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
futparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, df) in enumerate(futdata.items()):
    model, rcp = m.split(' ')
    sns.distplot(df.pdiff, ax=ax[i], kde=False, norm_hist=True)
    popt = stats.lognorm.fit(df.pdiff, loc=0, scale=1)
    fitline = stats.lognorm.pdf(np.arange(0, 101), *popt, )
    futparams = futparams.append({'Model':model, 'RCP':rcp,
                                  'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                  ignore_index=True)
    ax[i].plot(np.arange(0,101), fitline, color='r')
    ax[i].set_title("{0}\n({1:.4f}, {2:.4f}, {3:.4f})".format(m, *popt))
 

# Again, add the IBTrACS as reference
#sns.distplot(obstc.pdiff, fit=stats.lognorm, kde=False, color='red', ax=ax[-1])
obsparams = stats.lognorm.fit(obstc.pdiff)
#ax[-1].set_title("IBTrACS\n({0:.4f}, {1:.4f}, {2:.4f})".format(*obsparams))
fig.tight_layout()
plt.savefig(pjoin(path, "future_distribution.png"), bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), sharey=True)
x = np.arange(0, 100)

for index, row in futparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if row.Model == 'ENS':
        ax[0].plot(x, fitline, color='b')
    else:
        ax[0].plot(x, fitline, color='k')
    
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in futparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Projected')
    elif row.Model == 'ENS':
        ax[1].plot(x, fitline, color='b', label='Ensemble')
    else:
        ax[1].plot(x, fitline, color='k')
        
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None
plt.savefig(pjoin(path, 'uncorrected_distribution_by_RCP_future.png'), bbox_inches='tight')

Again, the simulated data is fitted to a lognormal distribution, and we can again see the simulated models (this time from the future time period) all have much lower distributions than the observed record.

In [ ]:
futparams.groupby('RCP').agg('mean')

When looking at the mean parameter fits for the two representative concentration pathways (RCPs), there is a larger different between the two, as the trend signal is different between the two RCPs.

Following are the $\delta_{fut}$ values for each individual model - i.e. the relationship between the quantiles in the reference period and the quantiles in the future period. It is here where we can see the range of outcomes across teh suite of models, and between the two selected emission scenarios. 

Some models (e.g. ACCESS1-0, MPI-ESM-LRQ) show little difference between reference and future quantiles, and also between the emission scenarios. The CSIRO-MK3.6 and GFDL-ESM2M display a consistent difference across emission scenarios, suggesting an absence of climate sensitivity (at least for $\Delta p_c$) in those models. The NorESM1-M displays an overall decline in intensity into the future. Counterintuitively, the ACCESS1-3 shows a stronger response in the RCP4.5 emission scenario compared to the RCP8.5, with a siginficant increase in the upper quantiles of $\Delta p_c$.

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()
x = np.linspace(0, 1, 101)
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')

    srefdata = refdf.pdiff.values
    sfutdata = futdf.pdiff.values
    srefq = np.quantile(srefdata, x)
    sfutq = np.quantile(sfutdata, x)
    ax[i].scatter(srefq, sfutq, alpha=0.5)
    ax[i].plot([0, 50], [0,50], '--', color='k', alpha=0.5)
    ax[i].set_title(m, size='small')
    ax[i].set_xlim((0, 50))
    ax[i].set_ylim((0, 50))
    ax[i].set_aspect('equal')
    
fig.tight_layout()
fig.text(0.5, 0.01, "Reference quantiles (hPa)", ha='center', va='center')
fig.text(0.01, 0.5, "Future quantiles (hPa)", ha='center', va='center', rotation='vertical')
None
plt.savefig(pjoin(path, 'ref_fut_quantiles.png'), bbox_inches='tight')

As a demonstration of the scaling process in practice, here we display the individual $\Delta p_c$ values for the reference and projection, as well as the observed $\Delta p_c$ values. On each, we add a line at the 99th percentile for each collection of data. The relative change in the $Q(0.99)$ between the reference period and the future period is 2.1. 

In [ ]:
refdf = refdata['ACCESS1-3Q RCP45']
futdf = futdata['ACCESS1-3Q RCP45']

refq099 = np.quantile(refdf.pdiff, 0.99)
futq099 = np.quantile(futdf.pdiff, 0.99)
obsq099 = np.quantile(obstc.pdiff, 0.99)
delta = futq099/refq099 
print(delta)
fig, ax = plt.subplots(1, 2, figsize=(16, 8), sharey=True)
ax[0].scatter(refdf.datetime, refdf.pdiff, s=1, c='r')
ax[0].scatter(futdf.datetime, futdf.pdiff, s=1, c='r')
ax[0].set_xlim(datetime(1981, 1, 1), datetime(2100, 1, 1))
ax[0].axhline(refq099, xmin=0, xmax=0.833333, linestyle='--', color='0.5')
ax[0].axhline(refq099, xmin=0, xmax=0.25, linestyle='--', color='k')
ax[0].axhline(futq099, xmin=0.83333, xmax=1, linestyle='--', color='k')
ax[1].scatter(obstc.ISO_TIME, obstc.pdiff, s=1, c='b')
ax[1].axhline(obsq099, xmin=0, xmax=0.83333, linestyle='--', color='0.5')
ax[1].axhline(obsq099, xmin=0, xmax=0.3333, linestyle='--', color='k')
ax[1].axhline(obsq099*delta, xmin = 0.83333, xmax=1, linestyle='--', color='k')
ax[1].set_xlim(datetime(1981, 1, 1), datetime(2100, 1, 1))
ax[0].set_ylabel(r"$\Delta p_c$ hPa")
ax[0].set_xlabel("Year")
ax[1].set_xlabel("Year")
ax[0].set_title('Modelled')
ax[1].set_title('Observed')

styles = mpatches.ArrowStyle.get_styles()
def to_texstring(s):
    s = s.replace("<", r"$<$")
    s = s.replace(">", r"$>$")
    s = s.replace("|", r"$|$")
    return s

x=datetime(2080, 1, 1)
y = 0.5 * (refq099 + futq099)
tx=datetime(2060, 1, 1)
ty=0.5 * (refq099 + futq099)
stylename = '-['
ax[0].annotate(r'$\delta_{fut}$' + ' = {0:.2f}'.format(delta), (x, y),
               (tx, ty), xycoords='data',
                ha="right", va="center",
                size=1.4,
                arrowprops=dict(arrowstyle=stylename,
                                fc="b", ec="b",
                                connectionstyle="arc3,rad=-0.05",
                                ),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[0].annotate(r"$Q_{ref}(0.99)$" + " = {0:.1f}".format(refq099), 
               (datetime(1995, 1, 1), refq099),
               (datetime(2000, 1, 1), 3*refq099),
               ha="center", va="center", size=1.4,
               arrowprops=dict(arrowstyle='->',
                                fc="b", ec="b", shrinkB=1.5,
                                connectionstyle="arc3,rad=-0.0",
                                ),
               fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[0].annotate(r"$Q_{fut}(0.99)$" + " = {0:.1f}".format(futq099), 
               (datetime(2090, 1, 1), futq099),
               (datetime(2080, 1, 1), 2*futq099),
               ha="center", va="center", size=1.4,
               arrowprops=dict(arrowstyle='->',
                                fc="b", ec="b", shrinkB=1.5,
                                connectionstyle="arc3,rad=-0.0",
                                ),
               fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[1].annotate(r"$Q_{obs}(0.99)$" + " = {0:.1f}".format(obsq099),
              (datetime(1995, 1, 1), obsq099),
              (datetime(1995, 1, 1), 1.5*obsq099), 
              ha="center", va='center', size=1.4,
              arrowprops=dict(arrowstyle="->",
                             fc='b', ec='b', shrinkB=1.5,
                             connectionstyle='arc3,rad=-0.05'),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))

ax[1].annotate(r"$Q_{futb}(0.99)$" + " = {0:.1f}".format(obsq099 * delta),
              (datetime(2090, 1, 1), obsq099 * delta),
              (datetime(2060, 1, 1), 1.5*obsq099), 
              ha="center", va='center', size=1.4,
              arrowprops=dict(arrowstyle="->",
                             fc='b', ec='b', shrinkB=1.5,
                             connectionstyle='arc3,rad=0.5'),
              fontsize='x-small', color='b',
              bbox=dict(boxstyle="square", fc="w", alpha=0.5))
plt.savefig(pjoin(path, 'illustration_qdm.png'), bbox_inches='tight')
None

In [ ]:
print(refq099)
print(futq099)
print(obsq099)
print(obsq099*delta)

### Applying the QDM

We begin by running the QDM algorithm for the reference period. This should give $\delta = 1$, as the 'reference' and 'future' data are the same. This should simplify down to a regular quantile mapping for the distribution between observed and reference period $\Delta p_c$, and allow for a mapping that can bring the simulated reference period data to a similar distribution to the observed data.

In [ ]:
obsdata = obstc.pdiff.values[obstc.pdiff.values > 0]
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
brefdata = {}
brefparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')

    srefdata = refdf.pdiff.values[refdf.pdiff.values > 0]
    brefdata[m] = qdm(obsdata, srefdata, srefdata)
    
    popt = stats.lognorm.fit(brefdata[m], loc=0, scale=1)
    brefparams = brefparams.append({'Model':model, 'RCP':rcp,
                                    'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                   ignore_index=True)
    ax[i].scatter(srefdata, brefdata[m], alpha=0.5)
    ax[i].plot([-5, 100], [-5, 100], '--')
    ax[i].set_xlabel("raw-tclv (hPa)")
    ax[i].set_ylabel("corrected-tclv (hPa)")
    ax[i].set_title(m)
    ax[i].set_xlim((-5, 100))
    ax[i].set_ylim((-5, 200))
    
fig.tight_layout()

Now we apply the QDM algorithm to the future data to determine the bias corrections for each individual model. 

In [ ]:
obsdata = obstc.pdiff.values[obstc.pdiff.values > 0]
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True)
ax = axes.flatten()
bfutparams = pd.DataFrame(columns=['Model', 'RCP', 'mu', 'sigma', 'zeta'])
bfutdata = {}
for i, (m, refdf) in enumerate(refdata.items()):
    futdf = futdata[m]
    model, rcp = m.split(' ')
    srefdata = refdf.pdiff.values[refdf.pdiff.values > 0]
    sfutdata = futdf.pdiff.values[futdf.pdiff.values > 0]
    bfutdata[m] = qdm(obsdata, srefdata, sfutdata)
    popt = stats.lognorm.fit(bfutdata[m], loc=0, scale=1)
    bfutparams = bfutparams.append({'Model':model, 'RCP':rcp,
                                    'mu':popt[0], 'sigma':popt[1], 'zeta':popt[2]},
                                   ignore_index=True)
    ax[i].scatter(sfutdata, bfutdata[m], alpha=0.5)
    ax[i].plot([-5, 100], [-5, 100], '--')
    ax[i].set_xlabel("raw-tclv (hPa)")
    ax[i].set_ylabel("corrected-tclv (hPa)")
    ax[i].set_title(m)
    ax[i].set_xlim((-5, 100))
    ax[i].set_ylim((-5, 200))
    
fig.tight_layout()

Comparing the corrected reference and future period distributions, we observe a significant change in the distribution parameters. Differences between RCP4.5 and RCP8.5 simulations are not significant. 

In [ ]:
brefparams.groupby('RCP').agg('mean')

In [ ]:
bfutparams.groupby('RCP').agg('mean')

Here we compare the distributions of the reference and future $\Delta p_c$ values for each model. This is based on the uncorrected data.

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()

x = np.arange(0, 51)
for i, row in enumerate(refparams.itertuples()):
    m = row.Model
    r = row.RCP
    refpopt = (row.mu, row.sigma, row.zeta)
    futrow = futparams[(futparams.Model==m) & (futparams.RCP==r)]
    futpopt = (futrow.mu, futrow.sigma, futrow.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    ks, pv = stats.ks_2samp(refpdf, futpdf)
    ax[i].plot(x, refpdf, 'k', label='Reference')
    ax[i].plot(x, futpdf, 'r', label='Future')
    ax[i].plot(x, stats.lognorm.pdf(x, *obsparams), 'g', lw=1, label='IBTrACS')
    ax[i].set_title('{0} {1}\n({2:.4f} {3:.4f})'.format(m, r, ks, pv))
    if i==0: ax[i].legend(loc=1)
        
#ax[-1].plot(x, stats.lognorm.pdf(x, *obsparams))
#ax[-1].set_title("IBTrACS")
fig.tight_layout()
plt.savefig(pjoin(path, "uncorrected.png"), bbox_inches='tight')

Note the ACCESS1-3, CSIRO Mk 3.6.0, GFDL-ESM2M, HadGEM2 and MIROC5 models have a very narrow distribution. This collection of models also have a lower grouping of annual frequency in the reference period - around 13-14 TCLV's per year. The remaining models have a mean annual frequency around 20 in the reference period. 

Now repeat for the bias-corrected data. In these, the label includes the Kolmogorov-Smirnov (K-S) test statistic and corresponding p-value for a two-sided K-S test for the two distrbutions. A small test statistic, or large p-value, means we cannot reject the null hypothesis that the two samples are drawn from the same distribution.

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True)
ax = axes.flatten()

x = np.arange(0, 101)
for i, row in enumerate(brefparams.itertuples()):
    m = row.Model
    r = row.RCP
    refpopt = (row.mu, row.sigma, row.zeta)
    futrow = bfutparams[(bfutparams.Model==m) & (bfutparams.RCP==r)]
    futpopt = (futrow.mu, futrow.sigma, futrow.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    ks, pv = stats.ks_2samp(refpdf, futpdf)
    ax[i].plot(x, refpdf, 'k', label='Reference')
    ax[i].plot(x, futpdf, 'r', label='Future')
    ax[i].plot(x, stats.lognorm.pdf(x, *obsparams), 'g', lw=1, label='IBTrACS')
    ax[i].set_title('{0} {1}\n({2:.4f} {3:.4f})'.format(m, r, ks, pv))
    if i==0: ax[i].legend(loc=1 )
        
#ax[-1].plot(x, stats.lognorm.pdf(x, *obsparams))
#ax[-1].set_title("IBTrACS")
fig.tight_layout()
None
plt.savefig(pjoin(path, "corrected.png"), bbox_inches='tight')

There are a range of responses between the reference period and the future period. In some models (e.g. ACCESS 1.3 RCP4.5, NorESM1-M RCP4.5) there is virtually no difference in the distributions. In many cases, there is a shift towards higher $\Delta p_c$ values in the future period. In a small number (GFDL-ESM2M RCP8.5, CSIRO Mk3.6 RCP4.5), the $\Delta p_c$ distribution shifts towards lower values. The IBTrACS distribution is shown in the lower right corner for reference.  

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5), sharey=True)
x = np.arange(0, 101, 0.5)
for i, row in enumerate(brefparams.itertuples()):
    refpopt = (row.mu, row.sigma, row.zeta)
    refpdf = stats.lognorm.pdf(x, *refpopt)
    if i == 0:
        ax[0].plot(x, refpdf, 'k', label="Reference", alpha=0.5)
    else:
        ax[0].plot(x, refpdf, 'k', alpha=0.5)

for i, row in enumerate(bfutparams.itertuples()):
    futpopt =(row.mu, row.sigma, row.zeta)
    futpdf = stats.lognorm.pdf(x, *futpopt)
    if i == 0:
        ax[1].plot(x, futpdf, 'k', label="Projected", alpha=0.5)
    else:
        ax[1].plot(x, futpdf, 'k', alpha=0.5)
    
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='IBTrACS')
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='IBTrACS')    
ax[0].legend(loc=1)
ax[1].legend(loc=1)

ax[0].set_xlabel(r'$\Delta p_c$ (hPa)')
ax[1].set_xlabel(r'$\Delta p_c$ (hPa)')
ax[0].set_ylabel("Probability")
#ax[1].set_ylabel("Probability")
fig.tight_layout()
None
plt.savefig(pjoin(path, "corrected_distributions.png"), bbox_inches='tight')

The top panel shows the distribution of bias-corrected $\Delta p_c$ values for the reference period (1981-2010, black) and the distribution of the IBTrACS data (red). The corrected model distributions very closely line up to the observed distrbution, as is the intention of the algorithm. 

The lower panel shows the corresponding distributions for the future period bias-corrected $\Delta p_c$ values. There is a significant variation of the projected distributions of $\Delta p_c$. 

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12,6), sharey=True)
x = np.arange(0, 100)

for index, row in bfutparams[refparams['RCP']=='RCP45'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    ax[0].plot(x, fitline, color='k')
ax[0].plot(x, stats.lognorm.pdf(x, *obsparams), 'r')
ax[0].set_title("RCP4.5")


for index, row in bfutparams[refparams['RCP']=='RCP85'].iterrows():
    popt = (row.mu, row.sigma, row.zeta)
    fitline = stats.lognorm.pdf(x, *popt)
    if index == 1:
        ax[1].plot(x, fitline, color='k', label='Projected')
    else:
        ax[1].plot(x, fitline, color='k')
ax[1].plot(x, stats.lognorm.pdf(x, *obsparams), 'r', label='Observations')
ax[1].set_title("RCP8.5")
fig.tight_layout()
ax[0].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].set_xlabel(r"$\Delta p_c$ (hPa)")
ax[1].legend()
None
plt.savefig(pjoin(path, 'corrected_distribution_by_RCP_future.png'), bbox_inches='tight')

### Applying the correction to derive new $p_c$ values

In this section, we insert the updated $\Delta p_c$ values back into the source data. This will allow us to use the simulated data as an input to the hazard modelling stage.  

At the same time, we calculate an estimate of the maximum sustained wind speed $v_{max}$ for each record. $v_{max}$ is caluclated in the manner described in Holland (2008):

$ b_s = -4.4 \times 10^{-5} \Delta p^2 + 0.01\Delta p + 0.03 \dfrac{\partial p_c}{\partial t} - 0.014 \phi + 0.15 v_t^x +1.0 $

$ x = 0.6 ( \,1 - \dfrac{\Delta p}{215} ) \,$, and

$ v_m = ( \,\!\dfrac{b_s}{\rho e}\, \Delta p) \, ^{0.5} $

Air density, $\rho$, is derived from the virtual temperature and pressure in the region of maximum winds ($R_{mw}$). $v_t$ is the translation speed of the TC and $\phi$ is the absolute latitude in degrees.

In [ ]:
outputPath = "../data/tclv/tracks/corrected"
fileTemplate = "{0}_bc.dat"

In [ ]:
for i, (m, refdf) in enumerate(refdata.items()):
    refdf_pos = refdf[refdf['pdiff'] > 0]
    refdf_pos['pmin'] = refdf_pos['poci'] - brefdata[m]
    refdf_pos['pdiff'] = brefdata[m]
    refdf = calculateMaxWind(refdf_pos, 'datetime')
    refdata[m] = refdf
    fname = pjoin(outputPath, fileTemplate.format(m.replace(' ', '_')))
    refdf.to_csv(fname, sep=',', float_format="%.3f", index=False)

In [ ]:
sns.distplot(refdata['ENS RCP85']['vmax'])

In [ ]:
# Now for the future data. Need a different file template so we can include the time period

futfileTemplate = "{0}_{1}-{2}_bc.dat"
for i, (m, futdf) in enumerate(futdata.items()):
    futdf_pos = futdf[futdf.pdiff > 0]
    futdf_pos['pmin'] = futdf_pos['poci'] - bfutdata[m]
    futdf_pos['pdiff'] = bfutdata[m]
    futdf = calculateMaxWind(futdf_pos, 'datetime')
    futdata[m] = futdf
    fname = pjoin(outputPath, futfileTemplate.format(m.replace(' ', '_'), start, end))
    futdf.to_csv(fname, sep=',',  float_format="%.3f", index=False)

In [ ]:
obstc=calculateMaxWind(obstc, 'ISO_TIME')
obstc['category'] = pd.cut(obstc['vmax'], [0, 25, 35, 46, 62, 77, 200], labels=labels)
sns.distplot(obstc['vmax'])

Following are histograms of TC intensity categories for each model, as well as the observed record (lower right panel). Once again there is close agreement between the models for the reference period (1981-2010), both between models and against the observed record. There is a discrepancy between the models and the observed for the 'TD' category (tropical depression, < 17 m/s), with the models indicating a lower proportion of these events. This is likely tied back to the decision to filter short-lived TCLV events. 

__NOTE__: These histograms are the proportion of TCLV records that are in each category. This does not give an indication of the _frequency_ of different category wind speeds. Each model has a unique change in total TCLV frequency that would need to be applied to this distribution to determine frequency of e.g. category 5 TCs.

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True, sharey=True)
ax = axes.flatten()
for i, (m, df) in enumerate(refdata.items()):
    df['category'] = pd.cut(df['vmax'], [0, 25, 35, 46, 62, 77, 200], labels=labels)
    x = df['category'].value_counts()/len(df)
    ax[i].bar(labels, x, color=catpal)
    ax[i].set_title("{0}".format(m))
    ax[i].set_xlabel('')
 

# Add the IBTrACS as reference
#x = obstc['category'].value_counts() / len(obstc)
#ax[-1].bar(labels, x, color=catpal)
#ax[-1].set_title("IBTrACS")
fig.suptitle("Reference period", size=20)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.savefig(pjoin(path, "categories_reference.png"), bbox_inches='tight')

The next figure shows the same distributions of TC intensity category for the future period. Here we note significant deviations between the models, as well as differences compared to the observed record. 

In [ ]:
fig, axes = plt.subplots(6,4, figsize=(20,20), sharex=True, sharey=True)
ax = axes.flatten()
labels = ['TD', 'TC1', 'TC2', 'TC3', 'TC4', 'TC5']
for i, (m, df) in enumerate(futdata.items()):
    df['category'] = pd.cut(df['vmax'], [0, 25, 35, 46, 62, 77, 200], labels=labels)
    x = df['category'].value_counts()/len(df)
    ax[i].bar(labels, x, color=catpal)
    ax[i].set_title("{0}".format(m))
    ax[i].set_xlabel('')
 

# Add the IBTrACS as reference
#x = obstc['category'].value_counts() / len(obstc)
#ax[-1].bar(labels, x, color=catpal)
#ax[-1].set_title("IBTrACS")
fig.suptitle('{0}-{1}'.format(start, end), size=20)
fig.tight_layout()
fig.subplots_adjust(top=0.95)
plt.savefig(pjoin(path, "categories_future.png"), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(1,3, figsize=(20, 7), sharex=True, sharey=True)
ax = axes.flatten()
labels = ['TD', 'TC1', 'TC2', 'TC3', 'TC4', 'TC5']

df = futdata['ENS RCP45']
x = df['category'].value_counts()/len(df)
ax[0].bar(labels, x, color=catpal)
ax[0].set_title('ENS RCP45')
df = futdata['ENS RCP85']
x = df['category'].value_counts()/len(df)
ax[1].bar(labels, x, color=catpal)
ax[1].set_title('ENS RCP85')
x = obstc['category'].value_counts() / len(obstc)
ax[2].bar(labels, x, color=catpal)
ax[2].set_title("IBTrACS")
plt.savefig(pjoin(path, "ensemble_category_projections.png"), bbox_inches='tight')

In [ ]:
fig, axes = plt.subplots(6, 4, figsize=(20, 20), sharex=True, sharey=True,
                         subplot_kw={'projection': ccrs.PlateCarree()})
ax = axes.flatten()

for i, (m, df) in enumerate(futdata.items()):
    ax[i].coastlines(resolution='50m', color='black', linewidth=1)
    ax[i].add_feature(feature.BORDERS)
    ax[i].add_feature(feature.OCEAN, zorder=0)
    gl = ax[i].gridlines(linestyle=":")
    for k,v in df.groupby('num'):
        ax[i].plot(v['lon'], v['lat'])
        
    ax[i].add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                fill=False, edgecolor='k', linewidth=3,
                                transform=ccrs.PlateCarree(), zorder=100)
                    )
    
    ax[i].set_title(m)
None
plt.savefig(pjoin(path, "future_tracks.png"), bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, 
                       figsize=(8, 8),
                       subplot_kw={'projection': ccrs.PlateCarree()})

ax[0].coastlines(resolution='50m', color='black', linewidth=1)
ax[0].add_feature(feature.BORDERS)
ax[0].add_feature(feature.OCEAN, zorder=0)
gl = ax[0].gridlines(linestyle=":")
df = futdata['ENS RCP45']
for k,v in df.groupby('num'):
    ax[0].plot(v['lon'], v['lat'])
ax[0].add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                   fill=False, edgecolor='k', linewidth=3,
                                   transform=ccrs.PlateCarree(), zorder=100)
                )
ax[0].set_title("ENS RCP45")

ax[1].coastlines(resolution='50m', color='black', linewidth=1)
ax[1].add_feature(feature.BORDERS)
ax[1].add_feature(feature.OCEAN, zorder=0)
gl = ax[1].gridlines(linestyle=":")
df = futdata['ENS RCP85']
for k,v in df.groupby('num'):
    ax[1].plot(v['lon'], v['lat'])
ax[1].add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                   fill=False, edgecolor='k', linewidth=3,
                                   transform=ccrs.PlateCarree(), zorder=100)
                )
ax[1].set_title("ENS RCP85")

None
plt.savefig(pjoin(path, "ensemble_future_tracks.png"), bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True, sharey=True, 
                       figsize=(8, 8),
                       subplot_kw={'projection': ccrs.PlateCarree()})

ax[0].coastlines(resolution='50m', color='black', linewidth=1)
ax[0].add_feature(feature.BORDERS)
ax[0].add_feature(feature.OCEAN, zorder=0)
gl = ax[0].gridlines(linestyle=":")
df = refdata['ENS RCP45']
for k,v in df.groupby('num'):
    ax[0].plot(v['lon'], v['lat'])
ax[0].add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                   fill=False, edgecolor='k', linewidth=3,
                                   transform=ccrs.PlateCarree(), zorder=100)
                )
ax[0].set_title("ENS RCP45")

ax[1].coastlines(resolution='50m', color='black', linewidth=1)
ax[1].add_feature(feature.BORDERS)
ax[1].add_feature(feature.OCEAN, zorder=0)
gl = ax[1].gridlines(linestyle=":")
df = refdata['ENS RCP85']
for k,v in df.groupby('num'):
    ax[1].plot(v['lon'], v['lat'])
ax[1].add_patch(mpatches.Rectangle(xy=[135, -25], width=25, height=15,
                                   fill=False, edgecolor='k', linewidth=3,
                                   transform=ccrs.PlateCarree(), zorder=100)
                )
ax[1].set_title("ENS RCP85")

None
plt.savefig(pjoin(path, "ensemble_reference_tracks.png"), bbox_inches='tight')